In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'C:/Users/s_csmscox/jupyterSave/eye_blink/train'
validation_dir = 'C:/Users/s_csmscox/jupyterSave/eye_blink/valid'

batch_size = 50
img_size = 32

# train_datagen = ImageDataGenerator(rescale=1/255)

train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=20,  # 지정된 각도 범위내에서 임의로 원본 이미지를 회전
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,    #  1-수치 혹은 1+수치만큼 확대 및 축소
                                   )



validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size,img_size),                      
    batch_size=batch_size,     
    class_mode='categorical'                    
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_size,img_size),                       
    batch_size=batch_size,    
    class_mode='categorical'                    
)

Found 54332 images belonging to 2 classes.
Found 13585 images belonging to 2 classes.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.applications import VGG16

np.random.seed(42)

# 모델 구축

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same',
                 activation='relu',
                 input_shape=(img_size, img_size, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

model.add(Dense(4096, activation='relu', kernel_initializer='he_normal'))
#model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu', kernel_initializer='he_normal'))
#model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu', kernel_initializer='he_normal'))

model.add(Dense(2, activation='softmax'))

model.summary()

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy',
                metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 64)        3

In [3]:
early_stopping = EarlyStopping(patience = 5) # 조기종료 콜백함수 정의

# 데이터 개수 / batch_size
steps_per_epoch = 54332 / batch_size
val_steps = 13585 / batch_size

history = model.fit(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps=val_steps,
                    callbacks=[early_stopping],
                    verbose=1
                    )

Epoch 1/100
1087/1086 [==============================] - 36s 33ms/step - loss: 0.4236 - accuracy: 0.7999 - val_loss: 0.2318 - val_accuracy: 0.9145
Epoch 2/100
1087/1086 [==============================] - 35s 33ms/step - loss: 0.2712 - accuracy: 0.8910 - val_loss: 0.1510 - val_accuracy: 0.9474
Epoch 3/100
1087/1086 [==============================] - 35s 33ms/step - loss: 0.1946 - accuracy: 0.9247 - val_loss: 0.1102 - val_accuracy: 0.9591
Epoch 4/100
1087/1086 [==============================] - 35s 33ms/step - loss: 0.1501 - accuracy: 0.9428 - val_loss: 0.0892 - val_accuracy: 0.9672
Epoch 5/100
1087/1086 [==============================] - 35s 33ms/step - loss: 0.1265 - accuracy: 0.9529 - val_loss: 0.0859 - val_accuracy: 0.9708
Epoch 6/100
1087/1086 [==============================] - 35s 32ms/step - loss: 0.1138 - accuracy: 0.9582 - val_loss: 0.0684 - val_accuracy: 0.9754
Epoch 7/100
1087/1086 [==============================] - 35s 32ms/step - loss: 0.1025 - accuracy: 0.9621 - val_loss: 0

In [4]:
model.compile(optimizer=Adam(learning_rate=1e-7), loss='categorical_crossentropy',
                metrics=['accuracy'])

history = model.fit(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps=val_steps,
                    callbacks=[early_stopping],
                    verbose=1
                    )

Epoch 1/100
1087/1086 [==============================] - 36s 33ms/step - loss: 0.0464 - accuracy: 0.9835 - val_loss: 0.0402 - val_accuracy: 0.9852
Epoch 2/100
1087/1086 [==============================] - 35s 32ms/step - loss: 0.0446 - accuracy: 0.9837 - val_loss: 0.0396 - val_accuracy: 0.9854
Epoch 3/100
1087/1086 [==============================] - 35s 32ms/step - loss: 0.0441 - accuracy: 0.9841 - val_loss: 0.0397 - val_accuracy: 0.9854
Epoch 4/100
1087/1086 [==============================] - 35s 33ms/step - loss: 0.0440 - accuracy: 0.9842 - val_loss: 0.0394 - val_accuracy: 0.9856
Epoch 5/100
1087/1086 [==============================] - 35s 32ms/step - loss: 0.0438 - accuracy: 0.9843 - val_loss: 0.0393 - val_accuracy: 0.9859
Epoch 6/100
1087/1086 [==============================] - 36s 33ms/step - loss: 0.0445 - accuracy: 0.9841 - val_loss: 0.0392 - val_accuracy: 0.9858
Epoch 7/100
1087/1086 [==============================] - 36s 33ms/step - loss: 0.0431 - accuracy: 0.9844 - val_loss: 0

In [5]:
model.save('C:/Users/s_csmscox/jupyterSave/eye_blink/eye_blink_CNN.h5')